In [238]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

%matplotlib inline

In [239]:
from mpmath import euler
emc = float(euler)
gum_loc = 2
norm_loc = 4
def f_stat(x):
    c2 = (1 + np.pi ** 2 / 6 + emc ** 2)
    c1 = 2 * emc * (norm_loc - gum_loc)
    c0 = (norm_loc - gum_loc) ** 2
    return x[0] ** 2 * c2 - x[0] * c1 + c0

def gradf_stat(x):
    c1 = 2 * (1 + np.pi ** 2 / 6 + emc ** 2)
    c0 = 2 * emc * (norm_loc - gum_loc)
    return np.array([x[0] * c1 - c0])

In [240]:
def f_rosenbrock(x):
    x1, x2 = x
    return 100 * (x2 - x1**2)**2 + (1 - x1)**2

def gradf_rosenbrock(x):
    x1, x2 = x
    return np.array([400 * x1 * (x1**2 - x2) + 2 * (x1 - 1), 200 * (x2 - x1**2)])

# Barzilai-Borwein method
### The idea of the method

- Gradient descent: $x_{k+1} = x_k - \alpha_k f'(x_k)$, $\alpha_k = \arg \min\limits_{\alpha > 0} f(x_{k+1})$
- Newton's method: $x_{k+1} = x_k - (f''(x_k))^{-1} f'(x_k)$
- Approximation of hessian using diagonal matrix:

$$
\alpha_k f'(x_k) = \alpha_k I f'(x_k) = \left( \frac{1}{\alpha_k} I \right)^{-1} f'(x_k) \approx f''(x_k))^{-1} f'(x_k)
$$

$$x_{k+1} = x_k - \alpha_k f'(x_k)$$ 


## How to find $\alpha$
- For exact hessian
$$
f''(x_{k})(x_{k} - x_{k-1}) = f'(x_{k}) - f'(x_{k-1})
$$
- For approximation

$$
\alpha_k^{-1} s_{k-1} \approx y_{k-1}
$$


Let $s_k = x_{k+1} - x_k$ и $y_k = f'(x_{k+1}) - f'(x_k)$.

Methods to find $\alpha$:
- First method
    - Problem
    
    $$
    \min_{\beta} \|\beta s_{k-1} - y_{k-1} \|^2_2
    $$
    
    - Solution
    
    $$
    \alpha = \frac{1}{\beta} = \frac{s^{\top}_{k-1} s_{k-1}}{s^{\top}_{k-1} y_{k-1}}
    $$
- Second method
    - Problem
    
    $$
    \min_{\alpha} \| s_{k-1} - \alpha y_{k-1} \|^2_2
    $$
    
    - Solution
    
    $$
    \alpha = \frac{s^{\top}_{k-1} y_{k-1}}{y^{\top}_{k-1} y_{k-1}}
    $$

In [241]:
def bb_first(g, s):
    return g.dot(s) / g.dot(g)

def bb_second(g, s):
    return s.dot(s) / g.dot(s)

def func_conv(x, **kwargs):
    return np.linalg.norm(kwargs['f'](x) - kwargs['f_true'](x))

def grad_conv(x, **kwargs):
    return np.linalg.norm(kwargs['gradf'](x))

In [215]:
def QuasiNewton(f, gradf, x0, epsilon, num_iter,
                step_selection=bb_first, convergence=grad_conv, **kwargs):
    x_prev = x0.copy()
    iteration = 0
    opt_args = {"f": f, "gradf": gradf}
    opt_args.update(kwargs)
    conv_values = []
    timestamps = []
    start = timer()
    alpha = 1e-4
    while iteration < num_iter:
        current_grad = gradf(x_prev)
        if iteration != 0:
            g = current_grad - prev_grad
            alpha = step_selection(g, s)
        x_next = x_prev - alpha * current_grad
        conv_value = convergence(x_next, **opt_args)
        conv_values.append(conv_value)
        curr = timer()
        timestamps.append((curr - start) * 1000)
        if conv_value < epsilon:
            break
        prev_grad = current_grad
        s = x_next - x_prev
        x_prev = x_next
        iteration += 1

    end = timer()
    time = (end - start) * 1000
    result = {"x": x_prev, "conv_values": conv_values, 
              "num_iter": len(conv_values), "time": time, 
              "time_per_iter": time / len(conv_values), 
              "timestamps": timestamps}
    return result

In [242]:
n = 3000
m = 100
x0_log = np.zeros(n)
A = np.random.rand(m, n) * 10
f_log = lambda x: -np.sum(np.log(1 - A.dot(x))) - np.sum(np.log(1 - x*x))
gradf_log = lambda x: np.sum(A.T / (1 - A.dot(x)), axis=1) + 2 * x / (1 - np.power(x, 2))

In [271]:
result = QuasiNewton(f_stat, gradf_stat, [100], 10**-6, 1000, bb_first)

In [272]:
print('x: ', result['x'], '\nf(x): ', f_stat(result['x']), 
      '\nnum_iter: ', result['num_iter'], '\ntime: ', result['time'])

x:  [0.38763892] 
f(x):  3.552497790744026 
num_iter:  3 
time:  3.350432001752779


In [269]:
result = QuasiNewton(f_rosenbrock, gradf_rosenbrock, [1.2, 1], 10**-6, 1000, bb_first)

In [270]:
print('x: ', result['x'], '\nf(x): ', f_rosenbrock(result['x']), 
      '\nnum_iter: ', result['num_iter'], '\ntime: ', result['time'])

x:  [0.99999878 0.99999756] 
f(x):  1.4869273741524936e-12 
num_iter:  19 
time:  3.6685169907286763


In [264]:
result = QuasiNewton(f_log, gradf_log, x0_log, 10**-6, 1000, bb_first)

In [267]:
print('x: ', result['x'], '\nf(x): ', f_log(result['x']), 
      '\nnum_iter: ', result['num_iter'], '\ntime: ', result['time'])

x:  [-0.13925411 -0.13367188 -0.13696304 ... -0.1389387  -0.12880329
 -0.11484648] 
f(x):  -706.4609962046102 
num_iter:  25 
time:  134.4779089849908
